In [1]:
from __future__ import print_function

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import time

In [3]:
from ilqr.cost import QRCost, FiniteDiffCost
from ilqr.mujoco_dynamics import MujocoDynamics
from ilqr.mujoco_controller import iLQR, RecedingHorizonController
from ilqr.examples.cartpole import CartpoleDynamics
from ilqr.dynamics import constrain

from scipy.optimize import approx_fprime

import mujoco_py
from mujoco_py import MjViewer
import os

In [4]:
def on_iteration(iteration_count, xs, us, J_opt, accepted, converged):
    J_hist.append(J_opt)
    info = "converged" if converged else ("accepted" if accepted else "failed")
    final_state = xs[-1]
    print("iteration", iteration_count, info, J_opt)

In [5]:
xml_path = os.path.join('..', 'ilqr', 'xmls', 'half_cheetah.xml')
"""
State-Space (name/joint/parameter):
        0- rootx     slider      position (m)
        1- rootz     slider      position (m)
        2- rooty     hinge       angle (rad)
        3- bthigh    hinge       angle (rad)
        4- bshin     hinge       angle (rad)
        5- bfoot     hinge       angle (rad)
        6- fthigh    hinge       angle (rad)
        7- fshin     hinge       angle (rad)
        8- ffoot     hinge       angle (rad)
        9- rootx     slider      velocity (m/s)
        10- rootz     slider      velocity (m/s)
        11- rooty     hinge       angular velocity (rad/s)
        12- bthigh    hinge       angular velocity (rad/s)
        13- bshin     hinge       angular velocity (rad/s)
        14- bfoot     hinge       angular velocity (rad/s)
        15- fthigh    hinge       angular velocity (rad/s)
        16- fshin     hinge       angular velocity (rad/s)
        17- ffoot     hinge       angular velocity (rad/s)
"""

"""
Actuators (name/actuator/parameter):
        - bthigh    hinge       torque (N m)
        - bshin     hinge       torque (N m)
        - bfoot     hinge       torque (N m)
        - fthigh    hinge       torque (N m)
        - fshin     hinge       torque (N m)
        - ffoot     hinge       torque (N m)
"""

dynamics = MujocoDynamics(xml_path, frame_skip = 1, use_multiprocessing = True)
print('dt:', dynamics.dt)

Finished loading process 69190
Finished loading process 69191
Finished loading process 69192
Finished loading process 69193
Finished loading process 69194
Finished loading process 69195
Finished loading process 69196
Finished loading process 69197
Finished loading process 69198
Finished loading process 69199
Finished loading process 69200
dt: 0.01
Finished loading process 69201
Finished loading process 69202
Finished loading process 69203
Finished loading process 69204
Finished loading process 69205


In [6]:
def l(x, u, i):
    action_cost = np.square(u).sum()
    vel_cost = 10 * (x[9] - 4) ** 2
    steady_cost = 200 * (x[10] ** 2)
    return action_cost + vel_cost + steady_cost

cost2 = FiniteDiffCost(l, lambda x, i: l(x, np.array([0, 0, 0, 0, 0, 0]), i), 18, 6, use_multiprocessing = True)

Finished loading process 69206
Finished loading process 69207
Finished loading process 69208
Finished loading process 69209
Finished loading process 69210
Finished loading process 69211
Finished loading process 69212
Finished loading process 69213
Finished loading process 69214
Finished loading process 69215
Finished loading process 69216
Finished loading process 69217
Finished loading process 69218
Finished loading process 69219
Finished loading process 69220
Finished loading process 69221


In [7]:
N = 200
x0 = dynamics.get_state()
us_init = np.random.uniform(-1, 1, (N, dynamics.action_size))
#us_init = np.zeros((N, dynamics.action_size))
ilqr = iLQR(dynamics, cost2, N)
mpc = RecedingHorizonController(x0, ilqr)

In [ ]:
t0 = time.time()
J_hist = []
controls = mpc.control(us_init, step_size = 2, initial_n_iterations = 500, subsequent_n_iterations = 100, on_iteration = on_iteration)
us = []
for i in range(100):
    print('ITERATION', i, '\n')
    us.append(next(controls)[1])
print('time', time.time() - t0)
us = np.concatenate(us)

ITERATION 0 

iteration 0 accepted 43147.81238941285
iteration 1 failed 43147.81238941285
iteration 2 failed 43147.81238941285
iteration 3 failed 43147.81238941285
iteration 4 accepted 43143.16925394749
iteration 5 accepted 43130.59185420938
iteration 6 failed 43130.59185420942
iteration 7 accepted 43098.66003511636
iteration 8 accepted 42900.13307606256
iteration 9 accepted 42862.66358705783
iteration 10 accepted 36862.407724088946
iteration 11 accepted 33873.63384005492
iteration 12 accepted 29859.410391999507
iteration 13 accepted 25853.16211573926
iteration 14 accepted 25524.201350458028
iteration 15 accepted 24233.147507723195
iteration 16 accepted 23865.222404630145
iteration 17 accepted 22739.86588790731
iteration 18 accepted 21673.318930912228
iteration 19 accepted 20900.465184061595
iteration 20 accepted 20603.267347714038
iteration 21 accepted 20331.045468894845
iteration 22 accepted 20074.314951412704
iteration 23 accepted 20019.215539403267
iteration 24 accepted 19762.49588

iteration 199 accepted 10655.31742375868
iteration 200 accepted 10652.740753185044
iteration 201 accepted 10633.38624285156
iteration 202 accepted 10595.407573563696
iteration 203 accepted 10577.59721792886
iteration 204 accepted 10564.74174492158
iteration 205 accepted 10561.137226941575
iteration 206 accepted 10545.305664002888
iteration 207 accepted 10535.21724637193
iteration 208 accepted 10532.314900866872
iteration 209 accepted 10524.014713601562
iteration 210 accepted 10510.241577982168
iteration 211 accepted 10496.534699110802
iteration 212 accepted 10481.902958924205
iteration 213 accepted 10461.881289618734
iteration 214 accepted 10446.432073663258
iteration 215 accepted 10402.065144069249
iteration 216 accepted 10392.831307313525
iteration 217 accepted 10385.084682380648
iteration 218 accepted 10369.505977383073
iteration 219 accepted 10355.544158512279
iteration 220 accepted 10353.079206660455
iteration 221 accepted 10350.524236545865
iteration 222 accepted 10342.1673983100

iteration 398 accepted 9490.922759104706
iteration 399 accepted 9486.744695392541
iteration 400 accepted 9483.749249507406
iteration 401 accepted 9481.8074982726
iteration 402 accepted 9479.423194134524
iteration 403 accepted 9476.560629088328
iteration 404 accepted 9474.088767384541
iteration 405 accepted 9472.555485981635
iteration 406 accepted 9471.090945258122
iteration 407 accepted 9469.290455260269
iteration 408 accepted 9466.970023991313
iteration 409 accepted 9466.25663514926
iteration 410 accepted 9464.836973878006
iteration 411 accepted 9457.629758517076
iteration 412 accepted 9454.277404413831
iteration 413 accepted 9451.524601119636
iteration 414 accepted 9449.257319861232
iteration 415 accepted 9447.64079426554
iteration 416 accepted 9440.219026001114
iteration 417 accepted 9438.10384657221
iteration 418 accepted 9435.927295010466
iteration 419 accepted 9434.72545682536
iteration 420 accepted 9434.255034841213
iteration 421 accepted 9432.721761786395
iteration 422 failed 9

//anaconda3/lib/python3.7/site-packages/ilqr-0.2.0-py3.7.egg/ilqr/mujoco_controller.py:167: UserWarning: exceeded max regularization term


ITERATION 3 

iteration 0 failed 8178.685939737862
iteration 1 accepted 8178.361000652719
iteration 2 accepted 8177.828941938929
iteration 3 failed 8177.8289419389275
iteration 4 accepted 8177.161052465017
iteration 5 failed 8177.161052465019
iteration 6 failed 8177.161052465019
iteration 7 failed 8177.161052465019
iteration 8 failed 8177.161052465019
iteration 9 failed 8177.161052465019
iteration 10 accepted 8176.8003496794945
iteration 11 accepted 8175.433603246643
iteration 12 accepted 8174.427359284049
iteration 13 accepted 8174.165110205538
iteration 14 failed 8174.165110205538
iteration 15 accepted 8174.058423345439
iteration 16 failed 8174.058423345442
iteration 17 failed 8174.058423345442
iteration 18 failed 8174.058423345442
iteration 19 accepted 8174.0407447768575
iteration 20 failed 8174.040744776856
iteration 21 failed 8174.040744776856
iteration 22 accepted 8172.313769988646
iteration 23 accepted 8172.286688111998
iteration 24 failed 8172.286688111998
iteration 25 failed 8

iteration 10 accepted 5446.451964460748
iteration 11 accepted 5445.670129694566
iteration 12 accepted 5444.579904855882
iteration 13 accepted 5441.836823818916
iteration 14 accepted 5439.552005888388
iteration 15 accepted 5438.04538452225
iteration 16 accepted 5434.189482844792
iteration 17 accepted 5431.605131342907
iteration 18 accepted 5427.2586528212405
iteration 19 accepted 5418.069100348216
iteration 20 accepted 5415.759411707434
iteration 21 accepted 5414.0823759817185
iteration 22 accepted 5413.231269250483
iteration 23 accepted 5411.330591803746
iteration 24 accepted 5409.24776365011
iteration 25 accepted 5406.100172420193
iteration 26 accepted 5403.63622620431
iteration 27 accepted 5401.947736429802
iteration 28 accepted 5399.345484419128
iteration 29 accepted 5396.9022629647225
iteration 30 accepted 5396.787199249215
iteration 31 accepted 5394.479531628442
iteration 32 accepted 5392.551212488876
iteration 33 accepted 5390.780467962251
iteration 34 accepted 5388.710923083162


iteration 47 accepted 4792.639959180743
iteration 48 accepted 4791.780908109005
iteration 49 failed 4791.780908109004
iteration 50 failed 4791.780908109004
iteration 51 failed 4791.780908109004
iteration 52 failed 4791.780908109004
iteration 53 failed 4791.780908109004
iteration 54 accepted 4791.676974198836
iteration 55 accepted 4787.977792016794
iteration 56 accepted 4786.021617326078
iteration 57 accepted 4785.042656620247
iteration 58 accepted 4784.692748118205
iteration 59 accepted 4780.981431570037
iteration 60 accepted 4778.961697309053
iteration 61 accepted 4777.393334425245
iteration 62 accepted 4776.149942554691
iteration 63 accepted 4775.022990375884
iteration 64 accepted 4774.781646234646
iteration 65 accepted 4773.8421383767245
iteration 66 accepted 4772.296334957029
iteration 67 accepted 4766.2771293175965
iteration 68 accepted 4756.587907665376
iteration 69 accepted 4749.548070472051
iteration 70 accepted 4747.6716104769275
iteration 71 accepted 4747.473628311818
iterati

iteration 0 failed 4109.909537060126
iteration 1 accepted 4109.532409329062
iteration 2 failed 4109.532409329062
iteration 3 accepted 4108.71507798306
iteration 4 failed 4108.715077983058
iteration 5 failed 4108.715077983058
iteration 6 failed 4108.715077983058
iteration 7 failed 4108.715077983058
iteration 8 accepted 4108.494640252569
iteration 9 accepted 4108.418032074886
iteration 10 failed 4108.4180320748865
iteration 11 accepted 4108.279785298777
iteration 12 failed 4108.279785298779
iteration 13 failed 4108.279785298779
iteration 14 failed 4108.279785298779
iteration 15 accepted 4108.185766634016
iteration 16 accepted 4108.177970451143
iteration 17 failed 4108.177970451139
iteration 18 failed 4108.177970451139
iteration 19 failed 4108.177970451139
iteration 20 accepted 4107.658707654042
iteration 21 converged 4107.658446046865
ITERATION 24 

iteration 0 failed 4937.113861019724
iteration 1 failed 4937.113861019724
iteration 2 failed 4937.113861019724
iteration 3 failed 4937.11386

iteration 68 accepted 4148.134969070235
iteration 69 accepted 4126.414461853071
iteration 70 accepted 4114.51624252316
iteration 71 accepted 4103.6894679652405
iteration 72 accepted 4096.214342798059
iteration 73 accepted 4090.372293550788
iteration 74 accepted 4073.3147522348336
iteration 75 accepted 4061.021940327532
iteration 76 accepted 4049.574589204216
iteration 77 accepted 4041.679383126998
iteration 78 accepted 4023.187219898651
iteration 79 accepted 4011.1438782800064
iteration 80 accepted 4002.2202692712126
iteration 81 accepted 3998.3985667689135
iteration 82 accepted 3991.392602652066
iteration 83 accepted 3982.496538275137
iteration 84 accepted 3970.5091205524923
iteration 85 accepted 3958.7632944380894
iteration 86 accepted 3955.4217528916924
iteration 87 accepted 3945.3926060878393
iteration 88 accepted 3941.5941277894176
iteration 89 accepted 3923.027068620515
iteration 90 accepted 3911.252072694753
iteration 91 accepted 3901.093586616533
iteration 92 accepted 3895.6252

iteration 72 accepted 2919.2013980945353
iteration 73 accepted 2916.790351433421
iteration 74 accepted 2916.7339165100516
iteration 75 accepted 2916.2883349443946
iteration 76 accepted 2915.765798851461
iteration 77 accepted 2914.9615758529535
iteration 78 accepted 2914.3959523909652
iteration 79 accepted 2914.262413808596
iteration 80 accepted 2913.44061048452
iteration 81 accepted 2912.7210933718065
iteration 82 accepted 2912.1916772641976
iteration 83 accepted 2911.3857690557325
iteration 84 accepted 2911.015432759852
iteration 85 accepted 2905.8716679161257
iteration 86 accepted 2904.8275089840463
iteration 87 accepted 2903.0619343051476
iteration 88 accepted 2899.8100540474693
iteration 89 failed 2899.8100540474693
iteration 90 failed 2899.8100540474693
iteration 91 failed 2899.8100540474693
iteration 92 failed 2899.8100540474693
iteration 93 failed 2899.8100540474693
iteration 94 failed 2899.8100540474693
iteration 95 failed 2899.8100540474693
iteration 96 accepted 2898.521412456

In [11]:
viewer = MjViewer(dynamics.sim)
dynamics.sim.reset()
start = x0
start[0] += 3
dynamics.set_state(start)
print(dynamics.get_state())
for i in range(us.shape[0]):
    dynamics.step(us[i])
    viewer.render()
    print(dynamics.get_state()[1], dynamics.get_state()[10], dynamics.get_state()[9])
    

Creating window glfw
[9. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
-0.003376425524294128 -0.33764255242941277 -0.08507476157155656
-0.007225019059473805 -0.38485935351796774 -0.14522168897078194
-0.011229826606341298 -0.4004807546867494 -0.2329837797341284
-0.015382072494809204 -0.41522458884679053 -0.30370493105838914
-0.019693538745433614 -0.4311466250624411 -0.32793800541452
-0.02409905626519079 -0.4405517519757175 -0.2407724647397232
-0.02885246329703 -0.4753407031839208 -0.18190769440992008
-0.033510303017017296 -0.46578397199872934 -0.09129767607419129
-0.037955375368868456 -0.4445072351851158 0.008665335822199041
-0.042289019248199965 -0.433364387933151 0.11542413317912896
-0.04720504885570598 -0.4916029607506012 0.23390359782786105
-0.05382410005865261 -0.6619051202946631 0.47409545437089673
-0.06296072428830894 -0.9136624229656336 0.8161383681574204
-0.06400355208160667 -0.10428277932977259 1.1091961830490131
-0.06435590545551122 -0.03523533739045488 1.10635496478128

-0.15259336619601513 -0.045998749249036505 3.871818079461599
-0.15291448422409515 -0.03211180280800318 3.813353993141332
-0.15320922564309597 -0.029474141900081838 3.833273132522033
-0.1538720939434157 -0.06628683003197235 3.8950992869786036
-0.1545954228512522 -0.07233289078364911 3.9273147540986137
-0.15533491559142049 -0.07394927401682973 3.942213384390101
-0.15608070518342113 -0.0745789592000649 3.954764516454386
-0.15682098754140297 -0.07402823579818488 3.9779157433963954
-0.15756802653039387 -0.07470389889908854 4.007889801708709
-0.15835433885320718 -0.07863123228133288 4.011932301055611
-0.15908186535211188 -0.07275264989046942 3.9572523961800843
-0.15920208121965448 -0.012021586754259558 4.05937461313062
-0.1595228172198047 -0.03207360001502426 3.930160019696208
-0.15951282134880315 0.000999587100155415 3.827302328591687
-0.15905220319783062 0.04606181509725384 3.78241113851502
-0.15905253277326245 -3.295754318455302e-05 3.9389915081213998
-0.15983882633994242 -0.0786293566679

In [10]:
print(us)

0.009459733372342061
